## Fish Disease Knowledge Graph Creation

This notebook reads a csv file which is a dataset on fish diseases and symptoms associated, and creates a neo4j knowledge graph database using the relationship between the fish diseases and symptoms.# Install below Python-NEO4J interaction library
!pip install neo4j

In [1]:
# Install below Python-NEO4J interaction library
#!pip install neo4j

### Import library

In [45]:
## import library
from neo4j import GraphDatabase
from neo4j.exceptions import ServiceUnavailable
import logging
import pandas as pd

### Connect to neo4j 

In [46]:
# Initialize the graph DB 
graph = GraphDatabase.driver(
    "neo4j://localhost:7687",
    #auth=("neo4j", "*****")
)

# Delete all the nodes and relationships
query = (
        "MATCH (all_nodes)"
        "OPTIONAL MATCH (all_nodes)-[all_rels]->()"
        "DELETE all_nodes, all_rels"
    )
with graph.session() as session:
    result = session.run(query)

### Load Fish Disease dataset

In [47]:
filename = 'FishDiseases.csv'
folderpath = r'/home/jengwei/nus/1. Intelligent Reasoning Systems/1. Machine Reasoning/MVP Project/Dataset//'

disease_df = pd.read_csv(folderpath + filename, encoding = "ISO-8859-1", index_col='id')
disease_df

,Disease,Symptom1,Symptom2,Symptom3,Symptom4,Symptom5,Symptom6,Symptom7,Symptom8,Symptom9,Symptom10,Symptom11,Symptom12,Causes,Treatments
id,,,,,,,,,,,,,,,
0,Hemorrhagic Septicemia,Distinct bright red streaks on fins,Patchy red discolorations on the flanks of the...,Bulging eyes,Bloated abdomens,List Open sores,Hyperventilation (fast breathing),Gasping at the surface,Erratic swimming,NaN,NaN,NaN,NaN,NaN,NaN
1,Columnaris,Frayed and ragged fins,Ulcerations on the skin,Epidermal loss,"White or cloudy, fungus-like patches. Particul...","Mucus on the gills, head, and dorsal regions",Gills will change color. Becoming light or dar...,Breathing rapidly and laboriously,Loss of appetite,NaN,NaN,NaN,NaN,NaN,NaN
2,Fish Tuberculosis,Your fish may become lethargic,Hollowed belly,Turn pale,Skin ulcers,Frayed fins,Fin and scale loss,Yellowish or darker nodules may appear on the ...,Loss of appetite,Refusing to eat,Hiding behavior,NaN,NaN,NaN,NaN
3,Dropsy,Bloated appearance,Scales stick out like a pinecone,Extruding eyes,Lethargic,Loss of appetite,Rapid gill breathing,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Bacterial Fin Rot,Milky white areas appearing in the fishs fins...,Shredded-looking and tattered fins decreasing ...,Exposed fin rays,Increasing difficulty swimming,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,Popeye,One or both eyes protruding,Cloudy eye,Bloodstained eye,Ruptured eye,Loss of vision,General signs of lethargy,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,Cloudy Eye,Cloudy white or gray haze over the eye(s),A loss of vision,May show signs of distress,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,Swim Bladder Disease,Swollen belly,Curved spine,Fish sinks to the bottom and struggles to rise up,Fish floats to the top upside down,Fish swims with its tail higher than its head,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,Ich,White spots on the body and gills,White spots may join together to form white pa...,Clamped fins,Labored breathing. Your fish may gasp for air ...,Loss of appetite,Your fish may rub up against objects in the tank,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Create Nodes and Relationship links in neo4j

In [48]:

diseases = []

for i in range(disease_df.shape[0]):
    ## traverse row-by-row
    # add nodes for disease
    query = (
        "MERGE (node: "+"Disease"+" {name: $name})"
        "RETURN node"
    )
    with graph.session() as session:
        result = session.run(query, name=disease_df.iloc[i,0])
    print("create new node with label as {0} and name as {1}".format("Disease", disease_df.iloc[i,0]))
    
    # add symptom nodes and relationship to disease
    for j in range(1, disease_df.shape[1]):
        # iterate across the associated symptoms  
        if pd.isnull(disease_df.iloc[i,j]) == False:
            # add symptom node
            query = (
                "MERGE (node: "+"Symptom"+" {name: $name})"
                "RETURN node"
            )
            with graph.session() as session:
                result = session.run(query, name=disease_df.iloc[i,j])
            print("create new node with label as {0} and name as {1}".format("Symptom", disease_df.iloc[i,j]))
            
            # add relationship
            query = (
                "MATCH (n1 {name: $name1})"
                "MATCH (n2 {name: $name2})"
                "MERGE (n1) - [r:"+"is_linked_to"+"] -> (n2)"   
                "RETURN n1, n2, r"
            )

            with graph.session() as session:
                result = session.run(query, name1=disease_df.iloc[i,j], name2=disease_df.iloc[i,0])
            print("create new relationship {0} - is_linked_to -> {1}".format(disease_df.iloc[i,j], disease_df.iloc[i,0]))


create new node with label as Disease and name as Hemorrhagic Septicemia 
create new node with label as Symptom and name as Distinct bright red streaks on fins
create new relationship Distinct bright red streaks on fins - is_linked_to -> Hemorrhagic Septicemia 
create new node with label as Symptom and name as Patchy red discolorations on the flanks of the body
create new relationship Patchy red discolorations on the flanks of the body - is_linked_to -> Hemorrhagic Septicemia 
create new node with label as Symptom and name as Bulging eyes
create new relationship Bulging eyes - is_linked_to -> Hemorrhagic Septicemia 
create new node with label as Symptom and name as Bloated abdomens
create new relationship Bloated abdomens - is_linked_to -> Hemorrhagic Septicemia 
create new node with label as Symptom and name as List Open sores
create new relationship List Open sores - is_linked_to -> Hemorrhagic Septicemia 
create new node with label as Symptom and name as Hyperventilation (fast breat

### Visualize created knowledge graph at: http://localhost:7474/browser/

* Username: neo4j

* Password: ai-user